In [3]:
import pyspark
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.sql import types

In [5]:
# Create SparkSession
spark = SparkSession.builder.master("local[*]") \
                    .appName('convert-to-parquet-application') \
                    .getOrCreate()

print(f'The PySpark {spark.version} version is running...')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/25 12:53:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/02/25 12:53:57 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/02/25 12:53:57 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


The PySpark 3.5.0 version is running...


In [12]:
df_yellow = spark.read\
     .option("header","true")\
     .csv('data/raw/yellow/2020/01/yellow_tripdata_2020_01.csv.gz')
df_yellow.schema

StructType([StructField('VendorID', StringType(), True), StructField('tpep_pickup_datetime', StringType(), True), StructField('tpep_dropoff_datetime', StringType(), True), StructField('passenger_count', StringType(), True), StructField('trip_distance', StringType(), True), StructField('RatecodeID', StringType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('PULocationID', StringType(), True), StructField('DOLocationID', StringType(), True), StructField('payment_type', StringType(), True), StructField('fare_amount', StringType(), True), StructField('extra', StringType(), True), StructField('mta_tax', StringType(), True), StructField('tip_amount', StringType(), True), StructField('tolls_amount', StringType(), True), StructField('improvement_surcharge', StringType(), True), StructField('total_amount', StringType(), True), StructField('congestion_surcharge', StringType(), True)])

In [14]:
df_green = spark.read\
     .option("header","true")\
     .csv('data/raw/green/2020/01/green_tripdata_2020_01.csv.gz')
df_green.schema

StructType([StructField('VendorID', StringType(), True), StructField('lpep_pickup_datetime', StringType(), True), StructField('lpep_dropoff_datetime', StringType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('RatecodeID', StringType(), True), StructField('PULocationID', StringType(), True), StructField('DOLocationID', StringType(), True), StructField('passenger_count', StringType(), True), StructField('trip_distance', StringType(), True), StructField('fare_amount', StringType(), True), StructField('extra', StringType(), True), StructField('mta_tax', StringType(), True), StructField('tip_amount', StringType(), True), StructField('tolls_amount', StringType(), True), StructField('ehail_fee', StringType(), True), StructField('improvement_surcharge', StringType(), True), StructField('total_amount', StringType(), True), StructField('payment_type', StringType(), True), StructField('trip_type', StringType(), True), StructField('congestion_surcharge', StringType()

In [18]:
green_schema = types.StructType([
    types.StructField("VendorID", types.IntegerType(), True),
    types.StructField("lpep_pickup_datetime", types.TimestampType(), True),
    types.StructField("lpep_dropoff_datetime", types.TimestampType(), True),
    types.StructField("store_and_fwd_flag", types.StringType(), True),
    types.StructField("RatecodeID", types.IntegerType(), True),
    types.StructField("PULocationID", types.IntegerType(), True),
    types.StructField("DOLocationID", types.IntegerType(), True),
    types.StructField("passenger_count", types.IntegerType(), True),
    types.StructField("trip_distance", types.DoubleType(), True),
    types.StructField("fare_amount", types.DoubleType(), True),
    types.StructField("extra", types.DoubleType(), True),
    types.StructField("mta_tax", types.DoubleType(), True),
    types.StructField("tip_amount", types.DoubleType(), True),
    types.StructField("tolls_amount", types.DoubleType(), True),
    types.StructField("ehail_fee", types.DoubleType(), True),
    types.StructField("improvement_surcharge", types.DoubleType(), True),
    types.StructField("total_amount", types.DoubleType(), True),
    types.StructField("payment_type", types.IntegerType(), True),
    types.StructField("trip_type", types.IntegerType(), True),
    types.StructField("congestion_surcharge", types.DoubleType(), True)
])


yellow_schema = types.StructType([
    types.StructField("VendorID", types.IntegerType(), True),
    types.StructField("tpep_pickup_datetime", types.TimestampType(), True),
    types.StructField("tpep_dropoff_datetime", types.TimestampType(), True),
    types.StructField("passenger_count", types.IntegerType(), True),
    types.StructField("trip_distance", types.DoubleType(), True),
    types.StructField("RatecodeID", types.IntegerType(), True),
    types.StructField("store_and_fwd_flag", types.StringType(), True),
    types.StructField("PULocationID", types.IntegerType(), True),
    types.StructField("DOLocationID", types.IntegerType(), True),
    types.StructField("payment_type", types.IntegerType(), True),
    types.StructField("fare_amount", types.DoubleType(), True),
    types.StructField("extra", types.DoubleType(), True),
    types.StructField("mta_tax", types.DoubleType(), True),
    types.StructField("tip_amount", types.DoubleType(), True),
    types.StructField("tolls_amount", types.DoubleType(), True),
    types.StructField("improvement_surcharge", types.DoubleType(), True),
    types.StructField("total_amount", types.DoubleType(), True),
    types.StructField("congestion_surcharge", types.DoubleType(), True)
])


In [63]:
def write_parquet(file, schema):
    years = [2020,2021]
    for year in years:
        for month in range(1,13):
            try:
                print(f'processing data for {file} trip data for {year}/{month:02d}')
                input_path = f'data/raw/{file}/{year}/{month:02d}'
                output_path = f'data/pq/{file}/{year}/{month:02d}'
                df = spark.read\
                     .option('header','True')\
                     .schema(schema)\
                     .csv(input_path)
                df.repartition(4).write.parquet(output_path, mode='overwrite')  
            except:  
                print('error occured')
    print('write to parquet completed for ',file)

In [65]:
write_parquet('yellow',yellow_schema)     

processing data for yellow trip data for 2020/01


processing data for yellow trip data for 2020/02


processing data for yellow trip data for 2020/03


processing data for yellow trip data for 2020/04


processing data for yellow trip data for 2020/05


...

processing data for yellow trip data for 2021/04


processing data for yellow trip data for 2021/05


processing data for yellow trip data for 2021/06


processing data for yellow trip data for 2021/07


processing data for yellow trip data for 2021/08
processing data for yellow trip data for 2021/09
error occured
processing data for yellow trip data for 2021/10
error occured
processing data for yellow trip data for 2021/11
error occured
processing data for yellow trip data for 2021/12
error occured
write to parquet completed for  yellow


In [67]:
write_parquet('green', green_schema)     

processing data for green trip data for 2020/01


processing data for green trip data for 2020/02


processing data for green trip data for 2020/03


processing data for green trip data for 2020/04
processing data for green trip data for 2020/05
processing data for green trip data for 2020/06
processing data for green trip data for 2020/07
processing data for green trip data for 2020/08


processing data for green trip data for 2020/09
processing data for green trip data for 2020/10
processing data for green trip data for 2020/11
processing data for green trip data for 2020/12
processing data for green trip data for 2021/01
processing data for green trip data for 2021/02
processing data for green trip data for 2021/03


processing data for green trip data for 2021/04


processing data for green trip data for 2021/05


processing data for green trip data for 2021/06


processing data for green trip data for 2021/07


processing data for green trip data for 2021/08
processing data for green trip data for 2021/09
error occured
processing data for green trip data for 2021/10
error occured
processing data for green trip data for 2021/11
error occured
processing data for green trip data for 2021/12
error occured
write to parquet completed for  green
